In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [21]:
df_chacha1_acc = pd.read_csv('TanzDaten\\chacha1\\Linear Acceleration.csv')
#df_chacha2 = pd.read_csv('TanzDaten\\chacha2\\Linear Acceleration.csv')
df_rumba1_acc = pd.read_csv('TanzDaten\\rumba1\\Linear Acceleration.csv')
df_jive1_acc = pd.read_csv('TanzDaten\\jive1\\Linear Acceleration.csv')
df_samba1_acc = pd.read_csv('TanzDaten\\samba1\\Linear Acceleration.csv')
#df_lwalzer1 = pd.read_csv('TanzDaten\\lwalzer1\\Linear Acceleration.csv')
#df_tango1 = pd.read_csv('TanzDaten\\tango1\\Linear Acceleration.csv')

df_chacha1_gyr = pd.read_csv('TanzDaten\\chacha1\\Gyroscope.csv')
df_rumba1_gyr = pd.read_csv('TanzDaten\\rumba1\\Gyroscope.csv')
df_jive1_gyr = pd.read_csv('TanzDaten\\jive1\\Gyroscope.csv')
df_samba1_gyr = pd.read_csv('TanzDaten\\samba1\\Gyroscope.csv')

df_acc_list = [df_chacha1_acc, df_rumba1_acc, df_jive1_acc, df_samba1_acc]
df_gyr_list = [df_chacha1_gyr, df_rumba1_gyr, df_jive1_gyr, df_samba1_gyr]
labels = ['chacha', 'rumba', 'jive', 'samba']

In [22]:
def prep_df_acc(df, label, bin_size=5):
    df.columns = ['time', 'x', 'y', 'z']

    bins = list(range(0, int(np.ceil(df['time'].max()+bin_size)), bin_size))
    df['bin'] = pd.cut(df['time'], bins=bins, labels=bins[0:-1])

    custom_agg = {
        'x': ['mean', 'max', 'min', 'var'],
        'y': ['mean', 'max', 'min', 'var'],
        'z': ['mean', 'max', 'min', 'var']
    }

    result = df.groupby('bin').agg(custom_agg).reset_index()
    #result.drop('bin', axis=1, inplace=True)
    result.columns = ['bin'] + [' acc '.join(col).strip() for col in result.columns.values[1:]]
    result['label'] = label

    return result

In [23]:
def prep_df_gyr(df, label, bin_size=5):
    df.columns = ['time', 'x', 'y', 'z']

    bins = list(range(0, int(np.ceil(df['time'].max()+bin_size)), bin_size))
    df['bin'] = pd.cut(df['time'], bins=bins, labels=bins[0:-1])

    custom_agg = {
        'x': ['mean', 'max', 'min', 'var'],
        'y': ['mean', 'max', 'min', 'var'],
        'z': ['mean', 'max', 'min', 'var']
    }

    result = df.groupby('bin').agg(custom_agg).reset_index()
    #result.drop('bin', axis=1, inplace=True)
    result.columns = ['bin'] + [' gyr '.join(col).strip() for col in result.columns.values[1:]]
    result['label'] = label

    return result

In [24]:
def feature_generator(df_acc_list, df_gyr_list, labels, bin_size=5):
    df_features_acc = pd.concat([prep_df_acc(df, label, bin_size=bin_size) for df, label in list(zip(df_acc_list, labels))], ignore_index=True)
    df_features_gyr = pd.concat([prep_df_gyr(df, label, bin_size=bin_size) for df, label in list(zip(df_gyr_list, labels))], ignore_index=True)

    df = pd.merge(df_features_acc, df_features_gyr, on=['bin', 'label'])
    df.drop('bin', axis=1, inplace=True)

    return df

def feature_generator(df_list, labels, bin_size=5):
    df = pd.concat([prep_df_acc(df, label, bin_size=bin_size) for df, label in list(zip(df_list, labels))], ignore_index=True)
    df.drop('bin', axis=1, inplace=True)

    return df
    

In [25]:
df_features = feature_generator(df_acc_list, labels, bin_size=10)

In [26]:
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [27]:
X = df_features[list(df_features.columns)[0:-1]]
#X = df_features.drop('label', axis=1)
y = df_features['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(42, 12)
(42,)
(19, 12)
(19,)


In [28]:
clf = tree.DecisionTreeClassifier(max_depth=3)
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=3)

In [29]:
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)
# more evaluation

0.631578947368421

In [ ]:
plt.figure(figsize=(12,12))  # set plot size (denoted in inches)
tree.plot_tree(clf, fontsize=10)
plt.show()